In [1]:
import numpy as np
import itertools
from scipy.stats import ks_2samp
from sklearn.neighbors import NearestNeighbors
rng = np.random.default_rng()
import pandas as pd
from dadapy import Data
from dadapy._utils import utils as ut
import os
import matplotlib.pyplot as plt
plt.style.use('ggplot')
import warnings
from sklearn.metrics import mean_absolute_error, mean_squared_error
from scipy.spatial import distance
import numpy as np

from sentence_transformers import SentenceTransformer, util
import random
import xml.etree.ElementTree as ET

import anthropic
import openai
import gc
from scipy import stats
import torch
import os
import json
import re

operouteapi = "..."
from openai import OpenAI

client = OpenAI(
  base_url="https://openrouter.ai/api/v1",
  api_key=operouteapi,
)

/Users/federico.ravenda/anaconda3/envs/EMNLP/lib/python3.10/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


In [2]:
RANDOM_SEED = 0
np.random.seed(RANDOM_SEED)
random.seed(RANDOM_SEED)
torch.manual_seed(RANDOM_SEED)

In [3]:
bdi_items = [
    # 1. Sadness
    [
        "I do not feel sad.",
        "I feel sad much of the time.",
        "I am sad all the time.",
        "I am so sad or unhappy that I can't stand it."
    ],
    # 2. Pessimism
    [
        "I am not discouraged about my future.",
        "I feel more discouraged about my future than I used to be.",
        "I do not expect things to work out for me.",
        "I feel my future is hopeless and will only get worse."
    ],
    # 3. Past Failure
    [
        "I do not feel like a failure.",
        "I have failed more than I should have.",
        "As I look back, I see a lot of failures.",
        "I feel I am a total failure as a person."
    ],
    # 4. Loss of Pleasure
    [
        "I get as much pleasure as I ever did from the things I enjoy.",
        "I don't enjoy things as much as I used to.",
        "I get very little pleasure from the things I used to enjoy.",
        "I can't get any pleasure from the things I used to enjoy."
    ],
    # 5. Guilty Feelings
    [
        "I don't feel particularly guilty.",
        "I feel guilty over many things I have done or should have done.",
        "I feel quite guilty most of the time.",
        "I feel guilty all of the time."
    ],
    # 6. Punishment Feelings
    [
        "I don't feel I am being punished.",
        "I feel I may be punished.",
        "I expect to be punished.",
        "I feel I am being punished."
    ],
    # 7. Self-Dislike
    [
        "I feel the same about myself as ever.",
        "I have lost confidence in myself.",
        "I am disappointed in myself.",
        "I dislike myself."
    ],
    # 8. Self-Criticalness
    [
        "I don't criticize or blame myself more than usual.",
        "I am more critical of myself than I used to be.",
        "I criticize myself for all of my faults.",
        "I blame myself for everything bad that happens."
    ],
    # 9. Suicidal Thoughts or Wishes
    [
        "I don't have any thoughts of killing myself.",
        "I have thoughts of killing myself, but I would not carry them out.",
        "I would like to kill myself.",
        "I would kill myself if I had the chance."
    ],
    # 10. Crying
    [
        "I don't cry anymore than I used to.",
        "I cry more than I used to.",
        "I cry over every little thing.",
        "I feel like crying, but I can't."
    ],
    # 11. Agitation
    [
        "I am no more restless or wound up than usual.",
        "I feel more restless or wound up than usual.",
        "I am so restless or agitated that it's hard to stay still.",
        "I am so restless or agitated that I have to keep moving or doing something."
    ],
    # 12. Loss of Interest
    [
        "I have not lost interest in other people or activities.",
        "I am less interested in other people or things than before.",
        "I have lost most of my interest in other people or things.",
        "It's hard to get interested in anything."
    ],
    # 13. Indecisiveness
    [
        "I make decisions about as well as ever.",
        "I find it more difficult to make decisions than usual.",
        "I have much greater difficulty in making decisions than I used to.",
        "I have trouble making any decisions."
    ],
    # 14. Worthlessness
    [
        "I do not feel I am worthless.",
        "I don't consider myself as worthwhile and useful as I used to.",
        "I feel more worthless as compared to other people.",
        "I feel utterly worthless."
    ],
    # 15. Loss of Energy
    [
        "I have as much energy as ever.",
        "I have less energy than I used to have.",
        "I don't have enough energy to do very much.",
        "I don't have enough energy to do anything."
    ],
    # 16. Changes in Sleeping Pattern
    [
        "I have not experienced any change in my sleeping pattern.",
        "I sleep somewhat more than usual OR I sleep somewhat less than usual.",
        "I sleep a lot more than usual OR I sleep a lot less than usual.",
        "I sleep most of the day OR I wake up 1-2 hours early and can't get back to sleep."
    ],
    # 17. Irritability
    [
        "I am no more irritable than usual.",
        "I am more irritable than usual.",
        "I am much more irritable than usual.",
        "I am irritable all the time."
    ],
    # 18. Changes in Appetite
    [
        "I have not experienced any change in my appetite.",
        "My appetite is somewhat less than usual OR My appetite is somewhat greater than usual.",
        "My appetite is much less than before OR My appetite is much greater than usual.",
        "I have no appetite at all OR I crave food all the time."
    ],
    # 19. Concentration Difficulty
    [
        "I can concentrate as well as ever.",
        "I can't concentrate as well as usual.",
        "It's hard to keep my mind on anything for very long.",
        "I find I can't concentrate on anything."
    ],
    # 20. Tiredness or Fatigue
    [
        "I am no more tired or fatigued than usual.",
        "I get more tired or fatigued more easily than usual.",
        "I am too tired or fatigued to do a lot of the things I used to do.",
        "I am too tired or fatigued to do most of the things I used to do."
    ],
    # 21. Loss of Interest in Sex
    [
        "I have not noticed any recent change in my interest in sex.",
        "I am less interested in sex than I used to be.",
        "I am much less interested in sex now.",
        "I have lost interest in sex completely."
    ]
]


sentences_bdi = list(itertools.chain.from_iterable(bdi_items)) 

bdis = []
for item in bdi_items:
    bd = ''
    for i, it in enumerate(item):
        bd = bd + str(i) + ' ' + it + '\n'
    bdis.append(bd)
    
items_names = ['Sadness', 'Pessimism', 'Past Failure', 'Loss of Pleasure', 'Guilty Feelings', 'Punishment Feelings', 
              'Self-Dislike', 'Self-Criticalness', 'Suicidal Thoughts or Wishes', 'Crying', 'Agitation', 
              'Loss of Interest', 'Indecisiveness', 'Worthlessness', 'Loss of Energy', 'Changes in Sleeping Pattern', 
              'Irritability', 'Changes in Appetite', 'Concentration Difficulty', 'Tiredness or Fatigue', 
              'Loss of Interest in Sex']

In [4]:
def parse_xml_with_recovery(file_path):
    texts = []
    try:
        with open(file_path, 'r') as file:
            lines = file.readlines()
        
        xml_string = ''
        for line in lines:
            try:

                ET.fromstring('<root>' + line.strip() + '</root>')
                xml_string += line
            except ET.ParseError:

                continue
        
        root = ET.fromstring('<root>' + xml_string + '</root>')
        texts = [text_elem.text for text_elem in root.findall('.//TEXT')]
    except Exception as e:
        print(f"Errore generale: {e}")
    
    return texts

def parse_xml_with_recovery(file_path):

    tree = ET.parse(file_path)
    root = tree.getroot()
    

    all_text = []
    

    text_elements = root.findall('.//TEXT')
    for text_elem in text_elements:
        if text_elem.text and text_elem.text != "/":
            all_text.append(text_elem.text)
    
    return "\n\n".join(all_text)


def transform_values(scores):
    
    scores_ = [str(s)[0] for s in scores]
    return np.array([1 if not s.isdigit() else s for s in scores_]).astype(np.float32)

def return_ids_kstar_binomial(data, embeddings, initial_id=None, Dthr=6.67, r='opt', n_iter = 10):
    if initial_id is None:
        data.compute_id_2NN(algorithm='base')
    else:
        data.compute_distances()
        data.set_id(initial_id)

    ids = np.zeros(n_iter)
    ids_err = np.zeros(n_iter)
    kstars = np.zeros((n_iter, data.N), dtype=int)
    log_likelihoods = np.zeros(n_iter)
    ks_stats = np.zeros(n_iter)
    p_values = np.zeros(n_iter)

    for i in range(n_iter):
      # compute kstar
      data.compute_kstar(Dthr)
      # print("iteration ", i)
      # print("id ", data.intrinsic_dim)

      # set new ratio
      r_eff = min(0.95,0.2032**(1./data.intrinsic_dim)) if r == 'opt' else r
      # compute neighbourhoods shells from k_star
      rk = np.array([dd[data.kstar[j]] for j, dd in enumerate(data.distances)])
      rn = rk * r_eff
      n = np.sum([dd < rn[j] for j, dd in enumerate(data.distances)], axis=1)
      # compute id
      id = np.log((n.mean() - 1) / (data.kstar.mean() - 1)) / np.log(r_eff)
      # compute id error
      id_err = ut._compute_binomial_cramerrao(id, data.kstar-1, r_eff, data.N)
      # compute likelihood
      log_lik = ut.binomial_loglik(id, data.kstar - 1, n - 1, r_eff)
      # model validation through KS test
      n_model = rng.binomial(data.kstar-1, r_eff**id, size=len(n))
      ks, pv = ks_2samp(n-1, n_model)
      # set new id
      data.set_id(id)

      ids[i] = id
      ids_err[i] = id_err
      kstars[i] = data.kstar
      log_likelihoods[i] = log_lik
      ks_stats[i] = ks
      p_values[i] = pv

    data.intrinsic_dim = id
    data.intrinsic_dim_err = id_err
    data.intrinsic_dim_scale = 0.5 * (rn.mean() + rk.mean())

    return ids, kstars[(n_iter - 1), :]#, ids_err, log_likelihoods, ks_stats, p_values

def find_Kstar_neighs(kstars, embeddings):
    nn = NearestNeighbors(metric = 'euclidean', n_jobs=-1)
    nn.fit(embeddings)

    neighs_ind = []
    for i, obs in enumerate(embeddings):
        distance, ind = nn.kneighbors([obs], n_neighbors=kstars[i] + 1)

        k_neighs = ind[0][1:]
        neighs_ind.append(k_neighs.tolist())
    return neighs_ind

def find_single_k_neighs(embeddings, index, k, cosine=True):
    target_embedding = embeddings[index]
    
    if cosine: 
        # Compute cosine distance
        all_distances = np.array([distance.cosine(target_embedding, emb) for emb in embeddings])
        # Sort by ascending order (smaller distance = higher similarity)
        nearest_indices = np.argsort(all_distances)[1:k+1]  
    else:
        # Compute dot product
        all_scores = util.dot_score(target_embedding, embeddings)[0].cpu().tolist()
        # Sort by descending order (larger score = higher similarity)
        nearest_indices = np.argsort(all_scores)[::-1][1:k+1]  

    return nearest_indices.tolist()
def extract_docs(results):

    unique_set = set()
    for sublist in results:
        unique_set.update(sublist)
    unique_docs = list(unique_set)

    return unique_docs

def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False

In [5]:
directory_path_neg = 'negative_examples_anonymous_chunks/chunk 1/'
files_neg = [f for f in os.listdir(directory_path_neg) if os.path.isfile(os.path.join(directory_path_neg, f))] #name of the files

ind_neg = [file.split('.')[0][:-2] for file in files_neg] 
ind_neg = [x for x in ind_neg if x != ""]

directory_path_pos = 'positive_examples_anonymous_chunks/chunk 1/'
files_pos = [f for f in os.listdir(directory_path_pos) if os.path.isfile(os.path.join(directory_path_pos, f))] #name of the files

ind_pos= [file.split('.')[0][:-2] for file in files_pos] 

In [6]:
directory_path_pos = 'positive_examples_anonymous_chunks/'
diz_pos = {}
for i in range(10):
    for subject in ind_pos:
        file_path = directory_path_pos + 'chunk ' + str(i+1) + '/' + subject + '_' + str(i+1) + '.xml'
        if subject not in diz_pos:
            diz_pos[subject] = []
        
        diz_pos[subject].append(parse_xml_with_recovery(file_path))

    
directory_path_neg = 'negative_examples_anonymous_chunks/'
diz_neg = {}
for i in range(10):
    for subject in ind_neg:
        
        file_path = directory_path_neg + 'chunk ' + str(i+1) + '/' + subject + '_' + str(i+1) + '.xml'

        if subject not in diz_neg:
            diz_neg[subject] = []
        
        diz_neg[subject].append(parse_xml_with_recovery(file_path))  

In [7]:

docs_pos = list(diz_pos.values())
docs_neg = list(diz_neg.values())
